In [16]:
def getDF(dataset_res):
    dataset_list = []
    for item in dataset_res['rows']:
        dataset_list.append(item['row'])
    return dataset_list        

In [17]:
import pandas as pd
from requests import HTTPError
from modules.clients import HuggingFaceClient

offset = 12376
hfc = HuggingFaceClient()
res = hfc.get(method='rows',params={
    'dataset' : "bigcode/commitpack",
    'config' : 'javascript',
    'split' : 'train',
    'offset' : offset,
    'length' : 100,
})

if res.status_code != 200:
    raise HTTPError(res.json())

df = pd.DataFrame(getDF(res.json()))
df[df['message'].str.contains('fix', case=False, na=False)]

,commit,old_file,new_file,old_contents,new_contents,subject,message,lang,license,repos
5,1d9c4818ff3a54bed6cfba2adff6d094304d972f,lib/doc/worksheet.js,lib/doc/worksheet.js,const _ = require('../utils/under-dash');\n\nc...,const _ = require('../utils/under-dash');\n\nc...,Fix sheet protection options,Fix sheet protection options\n\nSigned-off-by:...,JavaScript,mit,"guyonroche/exceljs,guyonroche/exceljs"
6,67a00e5b65d9785e72ef3fd7710825388dd303b4,src/components/MarkupItReactRenderer/index.js,src/components/MarkupItReactRenderer/index.js,"import React, { PropTypes } from 'react';\nimp...","import React, { PropTypes } from 'react';\nimp...",Fix rendering of plugins returining react comp...,Fix rendering of plugins returining react comp...,JavaScript,mit,"netlify/netlify-cms,netlify/netlify-cms,dopry/..."
11,89f50168ae4a8ae884cb9cae7a86184b02e114f4,common/modules/finder.jsm,common/modules/finder.jsm,// Copyright (c) 2008-2010 by Kris Maglione <m...,// Copyright (c) 2008-2010 by Kris Maglione <m...,Fix typo.,Fix typo.\n,JavaScript,mit,"madand/pentadactyl-pm,Kjwon15/dactyl,nathyong/..."
17,854622a523c9d49a7441c7f5611ef9b2d9b6eb6a,src/components/data-table-v2/data-table-v2.js,src/components/data-table-v2/data-table-v2.js,import settings from '../../globals/js/setting...,import settings from '../../globals/js/setting...,fix(data-table-v2): fix the undefined method c...,fix(data-table-v2): fix the undefined method c...,JavaScript,apache-2.0,"carbon-design-system/carbon-components,carbon-..."
25,bd0911637536cd0dc74d0f1fffb31fa9d86a87bb,lib/events-reader.js,lib/events-reader.js,"var _ = require('lodash'),\n _s = require('...","var _ = require('lodash'),\n _s = require('...",fixed a bug flow wasn't completing when there ...,fixed a bug\nflow wasn't completing when there...,JavaScript,mit,"michaelBenin/harvesterjs,agco/harvesterjs"
30,625d588dad8d833ae9e869b29f58a2aad1ab2466,src/components/media_control/media_control.js,src/components/media_control/media_control.js,// Copyright 2014 Globo.com Player authors. Al...,// Copyright 2014 Globo.com Player authors. Al...,"media control: bugfix, !!{} is true","media control: bugfix, !!{} is true\n",JavaScript,bsd-3-clause,"Peer5/clappr,shakin/clappr,Peer5/clappr,tarkan..."
37,11241069e52925372c4098a59dbd949a5d4682dd,packages/webpack-uni-pages-loader/lib/platform...,packages/webpack-uni-pages-loader/lib/platform...,"function generatePageCode (pages, pageOptions)...","function generatePageCode (pages, pageOptions)...",fix(v3): compatible with Android 4.4,fix(v3): compatible with Android 4.4\n,JavaScript,apache-2.0,"dcloudio/uni-app,dcloudio/uni-app,dcloudio/uni..."
43,a8686db58a481defdf831cfe3d5e7108e0e46d0b,src/containers/community/CommunitySettings.js,src/containers/community/CommunitySettings.js,import React from 'react'\nimport { connect } ...,import React from 'react'\nimport { connect } ...,fix linter and style warnings,fix linter and style warnings\n\nalso update t...,JavaScript,agpl-3.0,"Hylozoic/hylo-redux,Hylozoic/hylo-redux"
45,b81b7f067588764909415b0a9920afb90d1d47c6,lib/filter-engine.js,lib/filter-engine.js,"var \n\nasync = require(""async""),\npath = requ...","var \n\nasync = require(""async""),\npath = requ...","fix update-db-new and update-db-old, test passed","fix update-db-new and update-db-old, test pass...",JavaScript,mit,"cortexjs/cortex,rhgb/cortex,cortexjs/cortex,rh..."
54,d5e91319dd3bdba1af25c12b85ed66ff78e68baf,lib/fluentReports.js,lib/fluentReports.js,/*\n --------------------------------------\n ...,/*\n --------------------------------------\n ...,Initial Size Fixes,Initial Size Fixes\n,JavaScript,mit,"beaudesigns/fluentreports,Nathanaela/fluentrep..."


In [20]:
buggy_rows =  df[df['message'].str.contains('fix', case=False, na=False)]
only_code = buggy_rows[['old_contents','new_contents']].copy()
only_code.head()

,old_contents,new_contents
5,const _ = require('../utils/under-dash');\n\nc...,const _ = require('../utils/under-dash');\n\nc...
6,"import React, { PropTypes } from 'react';\nimp...","import React, { PropTypes } from 'react';\nimp..."
11,// Copyright (c) 2008-2010 by Kris Maglione <m...,// Copyright (c) 2008-2010 by Kris Maglione <m...
17,import settings from '../../globals/js/setting...,import settings from '../../globals/js/setting...
25,"var _ = require('lodash'),\n _s = require('...","var _ = require('lodash'),\n _s = require('..."


In [93]:
import sys
import code_diff as cd

def get_fix_commits(dataset_res: dict) -> pd.DataFrame:
    dataset_list = []
    for item in dataset_res['rows']:
        dataset_list.append(item['row'])
    df = pd.DataFrame(dataset_list)
    buggy_rows =  df[df['message'].str.contains('fix', case=False, na=False)]
    only_code = buggy_rows[['old_contents','new_contents']].copy()
    return only_code.to_dict(orient='records')



[{'old_contents': "const _ = require('../utils/under-dash');\n\nconst colCache = require('./../utils/col-cache');\nconst Range = require('./range');\nconst Row = require('./row');\nconst Column = require('./column');\nconst Enums = require('./enums');\nconst Image = require('./image');\nconst Table= require('./table');\nconst DataValidations = require('./data-validations');\nconst Encryptor = require('../utils/encryptor');\n\n// Worksheet requirements\n//  Operate as sheet inside workbook or standalone\n//  Load and Save from file and stream\n//  Access/Add/Delete individual cells\n//  Manage column widths and row heights\n\nclass Worksheet {\n  constructor(options) {\n    options = options || {};\n\n    // in a workbook, each sheet will have a number\n    this.id = options.id;\n    this.orderNo = options.orderNo;\n\n    // and a name\n    this.name = options.name || `Sheet${this.id}`;\n\n    // add a state\n    this.state = options.state || 'visible';\n\n    // rows allows access orga

<SStubPattern.SINGLE_TOKEN: 2>